In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import sys
import os

# Get the parent directory
parent_dir = os.path.abspath(os.path.join(os.path.dirname('.'), '..'))

# Add parent directory to sys.path
sys.path.append(parent_dir)

# Loading assets

In [5]:
from medrag.retrieval.vector_db import VectorDB
from medrag.retrieval.keyword_db import KeywordDB
from medrag.retrieval.reranking import rerank_documents
from medrag.summarization.summarization import QueryBasedTextRankSummarizer
from medrag.pipeline.qa_pipeline import QAPipeline

query_based_summarizer = QueryBasedTextRankSummarizer()
keyword_database = KeywordDB()
vector_database = VectorDB()

/home/mikhail/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
keyword_database.load_db()
vector_database.load_db()

2025-03-15 12:35:38.416 | INFO     | medrag.retrieval.vector_db:load_db:80 - vector database loaded from the local file successfully!


In [11]:
import torch
from langchain_huggingface import HuggingFaceEmbeddings

device = torch.device('cuda:0')
model = HuggingFaceEmbeddings(model_name="neuml/pubmedbert-base-embeddings", 
                                        model_kwargs={'device': device},
                                        encode_kwargs={"normalize_embeddings": True})

In [15]:
qa_pipeline = QAPipeline(vector_database, keyword_database, query_based_summarizer, embedding_model=model)

# Inference

In [13]:
import json
with open('/s3/misha/data_dir/PMC_patients/PMC-Patients-ReCDS/queries/train_queries.jsonl', 'r') as f:
    train_queries = [json.loads(line) for line in f]

In [16]:
import time

start_time = time.time()

# query = train_queries[0]['text']
query = 'A 44 year patient has tachycardia, unstable emotional condition and cries frequently.'

print(query)
print('-------------')
response = qa_pipeline.generate_response(query)

end_time = time.time()  # End the timer

elapsed_time = end_time - start_time
print(response)
print(f"Execution time: {elapsed_time:.4f} seconds")

A 44 year patient has tachycardia, unstable emotional condition and cries frequently.
-------------


Based on the provided medical records of similar patients, it appears that the patient in question is experiencing symptoms related to dissociative identity disorder (DID) and borderline personality disorder (BPD), which can manifest as unstable emotional conditions and frequent episodes of tachycardia. The treatment for these conditions often involves psychotropic medications.

Health recommendations for this patient:

1. **Stabilization of emotions**: Consider a medication with mood-stabilizing properties, such as lithium or anticonvulsants (e.g., valproate), which may help regulate emotional instability (Reference: [1] Case Report: Anomalous Experience in a Dissociative Identity and Borderline Personality Disorder).

2. **Tachycardia management**: Evaluate for underlying cardiac conditions that could be contributing to the tachycardia, such as cardiomyopathy or arrhythmias. Consider referral to a cardiologist or use of medications like beta blockers, which can help regulate heart ra

In [3]:
from deep_translator import GoogleTranslator
from langdetect import detect

# Initialize translator (Auto-detect source language)
translator = GoogleTranslator(source="auto", target="en")

# Translate a single text
text = "что такое ишемический инсульт?"
translated_text = translator.translate(text)
detected_lang = detect(text)

del translator
print("Translated:", translated_text)

Translated: What is ischemic stroke?
